In [5]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = gdf_edges.index
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

osmnx version 2.0.1


### Download network in a boundingbox

In [3]:
bounds = (18.029122582902115, 18.070836297501724, 59.33476653724975, 59.352622230576124)
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='./stockholm')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.5374898910522461 seconds ---


### Download network in a city

In [ ]:
place ="Stockholm, Sweden"
G = ox.graph_from_place(place, network_type='drive', which_result=2)
save_graph_shapefile_directional(G, filepath='stockholm')

### Download network with a boundary

In [ ]:
# Download by a boundary polygon in geojson
import osmnx as ox
from shapely.geometry import shape
json_file = open("stockholm_boundary.geojson")
import json
data = json.load(json_file)
boundary_polygon = shape(data["features"][0]['geometry'])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
save_graph_shapefile_directional(G, filepath='stockholm')

### Fix for Beijing issues

In [6]:
import osmnx as ox
import geopandas as gpd
import time

# Define place name
place_name = "Beijing, China"

# Download the drive network (routable)
G = ox.graph_from_place(place_name, network_type="drive")

# Convert the graph to GeoDataFrames (nodes and edges)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

# Define output directory
output_dir = "./beijing"
os.makedirs(output_dir, exist_ok=True)

# Save nodes and edges as ESRI Shapefiles
start_time = time.time()
gdf_nodes.to_file(f"{output_dir}/nodes.shp", encoding="utf-8")
gdf_edges.to_file(f"{output_dir}/edges.shp", encoding="utf-8")
print(f"Graph saved in {output_dir}")
print("--- %s seconds ---" % (time.time() - start_time))


/home/dieter/thesis/trace_obfuscate/.venv/lib/python3.12/site-packages/osmnx/_overpass.py:267: UserWarning: This area is 11 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
/tmp/ipykernel_52359/3184228518.py:20: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(f"{output_dir}/nodes.shp", encoding="utf-8")
/home/dieter/thesis/trace_obfuscate/.venv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(


Graph saved in ./beijing
--- 26.760839462280273 seconds ---


/home/dieter/thesis/trace_obfuscate/.venv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value '[157244803, 241099789, 241099798, 241099803, 241099815, 241099818, 241099823, 219958982, 219958998, 241100507, 241100508, 241100509, 241100512, 241100518, 241100519, 241100521, 241100522, 241100523, 241100526, 1201577967, 241100527, 241100529, 1027667059, 241100532, 1027667060, 241100534, 157091703, 241100537, 241100538]' of field osmid has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


add missing fid column


In [8]:
import geopandas as gpd

# Load the edges shapefile
edges = gpd.read_file("../osmnx_example/beijing/edges.shp")

# Add a unique 'fid' column
edges["fid"] = edges.index  # Unique index as ID

# Save the fixed shapefile
edges.to_file("../osmnx_example/beijing/edges.shp", encoding="utf-8")

print("Fixed shapefile saved.")

Fixed shapefile saved.


In [9]:
edges = gpd.read_file("../osmnx_example/beijing/edges.shp")
print(edges.columns)


Index(['u', 'v', 'key', 'osmid', 'highway', 'name', 'ref', 'oneway',
       'reversed', 'length', 'bridge', 'lanes', 'maxspeed', 'tunnel', 'access',
       'junction', 'width', 'service', 'fid', 'geometry'],
      dtype='object')
